In [ ]:
#from web3 import Web3

# mainnet DO NOT USE UNLESS FOR PRODUCTION
#infura_url = "https://mainnet.infura.io/v3/81e546ebd5c54cdd8f8488d75a1fa114" # replace 'mainnet' by 'sepolia' to use testnet
#w3 = Web3(Web3.HTTPProvider(infura_url))
#w3.is_connected()

True

In [2]:
# TEST WALLET CREATION

from web3 import Web3

# Generate a random private key
w3 = Web3()
private_key = w3.eth.account.create().key.hex()  # Generate a random private key

# Derive the wallet address
account = w3.eth.account.from_key(private_key)  # Use from_key instead of privateKeyToAccount

print(f"Your test wallet address: {account.address}")
print(f"Your test wallet private key: {private_key}")

#Your test wallet address: 0x804ef9814CB2f054A2cD772D598b6D54bc6105D6
#Your test wallet private key: a4e0be21933b4b7433caf8a4878e268ae1343c8468bc2fc449b802fe3d27882d

Your test wallet address: 0x6128Dc8A45283D626154d8553eC61ad4e243399f
Your test wallet private key: 21d9cac91489154631c2178d024ac568fc2e272dc17fae3a58200eeac57957f3


***

In [9]:
from web3 import Web3

# Connect to Sepolia
infura_url = "https://sepolia.infura.io/v3/81e546ebd5c54cdd8f8488d75a1fa114"
w3 = Web3(Web3.HTTPProvider(infura_url))

# Uniswap Factory contract on Sepolia
factory_contract_address = "0xF62c03E08ada871A0bEb309762E260a7a6a880E6"
factory_abi = [
    {
        "constant": True,
        "inputs": [
            {"name": "tokenA", "type": "address"},
            {"name": "tokenB", "type": "address"},
        ],
        "name": "getPair",
        "outputs": [{"name": "pair", "type": "address"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function",
    }
]

# Initialize the factory contract
factory_contract = w3.eth.contract(address=factory_contract_address, abi=factory_abi)

# Token addresses (replace with Sepolia ETH and USDT addresses)
eth_address = "0x7b79995e5f793A07Bc00c21412e50Ecae098E7f9"  # WETH
usdt_address = "0xaA8E23Fb1079EA71e0a56F48a2aA51851D8433D0"

# Fetch the pair address
pair_address = factory_contract.functions.getPair(eth_address, usdt_address).call()

print(f"ETH/USDT Pair Address: {pair_address}")

ETH/USDT Pair Address: 0x94Db6E5f082173EB15B6B098036E4Cdf80723Ad2


In [10]:
pair_abi = [
    {
        "constant": True,
        "inputs": [],
        "name": "getReserves",
        "outputs": [
            {"name": "_reserve0", "type": "uint112"},
            {"name": "_reserve1", "type": "uint112"},
            {"name": "_blockTimestampLast", "type": "uint32"},
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function",
    }
]

pair_contract = w3.eth.contract(address=pair_address, abi=pair_abi)
reserves = pair_contract.functions.getReserves().call()
print(f"Reserves: {reserves}")

Reserves: [0, 0, 0]


In [ ]:
from web3 import Web3
import time
from decimal import Decimal

# Connect to Sepolia Testnet via Infura
infura_url = "https://sepolia.infura.io/v3/81e546ebd5c54cdd8f8488d75a1fa114"
w3 = Web3(Web3.HTTPProvider(infura_url))

# ERC-20 ABI (basic interface to interact with tokens)
erc20_abi = [
    {
        "constant": True,
        "inputs": [],
        "name": "balanceOf",
        "outputs": [{"name": "", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function",
    },
    {
        "constant": False,
        "inputs": [
            {"name": "to", "type": "address"},
            {"name": "value", "type": "uint256"}
        ],
        "name": "transfer",
        "outputs": [{"name": "", "type": "bool"}],
        "payable": False,
        "stateMutability": "nonpayable",
        "type": "function",
    },
]

# Uniswap Router Contract Address on Sepolia (example)
uniswap_router_address = "0xF62c03E08ada871A0bEb309762E260a7a6a880E6"
uniswap_router_abi =   # Uniswap Router ABI here

# Set up wallet and contract
private_key = "21d9cac91489154631c2178d024ac568fc2e272dc17fae3a58200eeac57957f3"  # Ensure you keep your private key safe
my_address = "0x6128Dc8A45283D626154d8553eC61ad4e243399f"
router_contract = w3.eth.contract(address=uniswap_router_address, abi=uniswap_router_abi)

# Token addresses (replace with actual token addresses on Sepolia)
token1_address = "0xA0b86991C6218B36c1D19D4A2e9eb0Ce3606eB48"  # Example USDC address
token2_address = "0x7b79995e5f793A07Bc00c21412e50Ecae098E7f9"  # Example WETH address

# Define a simple function to get token balance
def get_token_balance(token_address):
    token_contract = w3.eth.contract(address=token_address, abi=erc20_abi)
    balance = token_contract.functions.balanceOf(my_address).call()
    return balance

# Define the function to swap tokens
def swap_tokens(amount_in, token_in, token_out):
    path = [token_in, token_out]
    amount_out_min = 0  # For simplicity, no slippage check
    to = my_address  # Send the tokens to your address
    deadline = int(time.time()) + 300  # 5-minute deadline for transaction
    txn = router_contract.functions.swapExactTokensForTokens(
        amount_in, amount_out_min, path, to, deadline
    ).buildTransaction({
        'from': my_address,
        'gas': 200000,
        'gasPrice': w3.toWei('20', 'gwei'),
        'nonce': w3.eth.getTransactionCount(my_address),
    })
    signed_txn = w3.eth.account.sign_transaction(txn, private_key)
    txn_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
    return txn_hash

# Main loop to perform trades
def trade():
    while True:
        balance_token1 = get_token_balance(token1_address)
        balance_token2 = get_token_balance(token2_address)

        # Naive strategy: if we have more token1, trade it for token2
        if balance_token1 > 0:
            amount_in = balance_token1 // 2  # Swap 50% of token1 balance
            txn_hash = swap_tokens(amount_in, token1_address, token2_address)
            print(f"Swapping {amount_in} Token1 for Token2, Txn Hash: {txn_hash.hex()}")

        # If we have more token2, trade it for token1
        elif balance_token2 > 0:
            amount_in = balance_token2 // 2  # Swap 50% of token2 balance
            txn_hash = swap_tokens(amount_in, token2_address, token1_address)
            print(f"Swapping {amount_in} Token2 for Token1, Txn Hash: {txn_hash.hex()}")

        # Wait for a minute before the next trade
        time.sleep(60)

# Start the bot
if __name__ == "__main__":
    trade()

InvalidAddress: ('Address has an invalid EIP-55 checksum. After looking up the address from the original source, try again.', '0xA0b86991C6218B36c1D19D4A2e9eb0Ce3606eB48')